In [2]:
import os
os.chdir("../")
%pwd

'c:\\Users\\shaur\\Desktop\\Glean_Implementation'

Bash commands

set MLFLOW_TRACKING_URI=https://dagshub.com/shauryat1298/Glean-Document-Understanding.mlflow

set MLFLOW_TRACKING_USERNAME=shauryat1298

set MLFLOW_TRACKING_PASSWORD=c5677632360a9e2d8981b4b2bf74d353830f2bc3

python script.py

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/shauryat1298/Glean-Document-Understanding.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "shauryat1298"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "c5677632360a9e2d8981b4b2bf74d353830f2bc3"

In [7]:
from src.Glean.config.configuration import ConfigurationManager
from src.Glean.components.train import TrainModel
from network import dataset
from pathlib import Path
import mlflow
import mlflow.pytorch
from urllib.parse import urlparse
import torch
from torch.utils import data
from sklearn.metrics import recall_score, precision_score, f1_score

In [4]:
def evaluate(model, val_dataloader, criterion):

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    val_accuracy = 0.0
    val_loss = 0.0
    y_preds = []
    y_labels = []

    with torch.no_grad():
        for val_field, val_candidate, val_words, val_positions, masks, val_labels in val_dataloader:
            val_field = val_field.to(device)
            val_candidate = val_candidate.to(device)
            val_words = val_words.to(device)
            val_positions = val_positions.to(device)
            masks = masks.to(device)
            val_labels = val_labels.to(device)

            val_outputs = model(val_field, val_candidate, val_words, val_positions, masks)
            validation_loss = criterion(val_outputs, val_labels)

            val_preds = val_outputs.round()
            y_preds.extend(list(val_preds.cpu().detach().numpy().reshape(1, -1)[0]))
            y_labels.extend(list(val_labels.cpu().detach().numpy().reshape(1, -1)[0]))

            val_accuracy += torch.sum(val_preds == val_labels).item()
            val_loss += validation_loss.item()

        val_loss = val_loss / val_dataloader.sampler.num_samples
        val_accuracy = val_accuracy / val_dataloader.sampler.num_samples
        recall = recall_score(y_labels, y_preds)
        precision = precision_score(y_labels, y_preds)
        f1 = f1_score(y_labels, y_preds)

    return val_accuracy, val_loss, recall, precision, f1

In [9]:
config = ConfigurationManager()
config = config.train_model_config()

doc_data = dataset.DocumentsDataset(config=config, split_name='val')
VOCAB_SIZE = len(doc_data.vocab)
test_data = data.DataLoader(doc_data, batch_size=config.batch_size, shuffle=True)

# rlie = Model(VOCAB_SIZE, config.EMBEDDING_SIZE, config.NEIGHBOURS, config.HEADS)
criterion = torch.nn.BCELoss()
relie = torch.load(Path(config.best_model_dir)/"model.pth")
# criterion = FocalLoss(alpha=2, gamma=5)

test_accuracy, test_loss, test_recall, test_precision, test_f1 = evaluate(relie, test_data, criterion)

mlflow.set_registry_uri(config.mlflow_uri)
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

with mlflow.start_run():
    mlflow.log_params(config.all_params)
    mlflow.log_metrics({
        "Test Accuracy": test_accuracy,
        "Test Loss": test_loss,
        "Test Recall": test_recall,
        "Test Precision": test_precision,
        "Test F1": test_f1
    })
    if tracking_url_type_store != "file":
        mlflow.pytorch(relie, "model", registered_model_name="TransformerModel")
    else:
        mlflow.pytorch.log_model(relie, "")
print(f"Test Accuracy: {test_accuracy} Test Loss: {test_loss} Test Recall: {test_recall} Test Precision {test_precision} Test F1 {test_f1}")

[2023-11-04 03:32:11,998: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-04 03:32:12,000: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-04 03:32:12,000: INFO: common: created directory at: artifacts]
[2023-11-04 03:32:12,001: INFO: common: created directory at: artifacts/train_model]
[2023-11-04 03:32:12,002: INFO: dataset: Preprocessed data available, Loading data from cache...]

Class Mapping: {'registration_num': 0}
Classs counts: {'registration_num': 99}
Test Accuracy: 0.8125904486251809 Test Loss: 0.12127573110063644 Test Recall: 0.8787878787878788 Test Precision 0.26047904191616766 Test F1 0.4018475750577367
